# Libraries

In [ ]:
import numpy as np
import pandas as pd

import requests, bs4
from datetime import datetime
import math

from IPython.display import clear_output

# Data

In [ ]:
dataset = pd.read_csv('data.csv')

# Function Definitons

In [ ]:
to_datetime = lambda x : datetime(int(x[:4]), int(x[5:7]), int(x[8:]))

In [ ]:
def websiteInfo(website, label):

    website_details = {'URL': [website],
                      'Label': [label]}
    try:
        url = 'https://www.whois.com/whois/'+website
        res = requests.get(url)
        if res.status_code == requests.codes.ok:
            ressoup = bs4.BeautifulSoup(res.text, 'lxml')
            elems = ressoup.select('.df-value')
            try:
                regOn = to_datetime(elems[2].getText())
                expOn = to_datetime(elems[3].getText())
                duration = (expOn - regOn).total_seconds()//31556926
                website_details['domainRegistrationLength'] = [int(duration)]
            except Exception as e:
                website_details['domainRegistrationLength'] = [e]
        else:
            website_details['domainRegistrationLength'] = [-1]
    except Exception as e:
        website_details['domainRegistrationLength'] = [e]
        
    try:
        url = 'https://www.alexa.com/siteinfo/'+website
        res = requests.get(url)
        if res.status_code == requests.codes.ok:
            ressoup = bs4.BeautifulSoup(res.text, 'lxml')

            elems = ressoup.select('.num.purple')
            try:
                website_details['Search Traffic'] = [elems[0].getText()]
            except Exception as e:
                website_details['Search Traffic'] = [e]
            try:
                website_details['Bounce Rate'] = [elems[1].getText()]
            except Exception as e:
                website_details['Bounce Rate'] = [e]

            elems = ressoup.select('.big.data')
            try:
                website_details['Total Sites Linking in'] = [elems[1].getText()]
            except Exception as e:
                website_details['Total Sites Linking in'] = [e]

            elems = ressoup.select('.rankmini-rank')
            try:
                website_details['Alexa Rank'] = [elems[0].getText().strip()]
            except Exception as e:
                website_details['Alexa Rank'] = [e]

            elems = ressoup.select('.small.data')
            try:
                website_details['Daily Views per Visitor'] = [elems[1].getText().strip().split()[0]]
            except Exception as e:
                website_details['Daily Views per Visitor'] = [e]
            try:
                website_details['Daily Time on Site'] = [elems[2].getText().strip().split()[0]]
            except Exception as e:
                website_details['Daily Time on Site'] = [e]
        else:
            website_details['Search Traffic'] = [-1]
            website_details['Bounce Rate'] = [-1]
            website_details['Total Sites Linking in'] = [-1]
            website_details['Alexa Rank'] = [-1]
            website_details['Daily Views per Visitor'] = [-1]
            website_details['Daily Time on Site'] = [-1]
    except Exception as e:
        website_details['Search Traffic'] = [e]
        website_details['Bounce Rate'] = [e]
        website_details['Total Sites Linking in'] = [e]
        website_details['Alexa Rank'] = [e]
        website_details['Daily Views per Visitor'] = [e]
        website_details['Daily Time on Site'] = [e]
    
    print(website_details)
    website_details = pd.DataFrame(website_details)

    return website_details

# Scraping

In [ ]:
i = 1
for x, y in zip(dataset['URL'], dataset['Label']):
    print(i)
    w = websiteInfo(x,y)
    w.to_json('Metrics/'+str(i)+'.json')
    print('-'*50)
    i+=1